# Dataset Versioning and Endpoint Deployment

<a href="https://colab.research.google.com/github/VertaAI/modeldb/blob/master/client/workflows/demos/dataset-registry-endpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook combines the concepts of [census-with-new-endpoints.ipynb](census-with-new-endpoints.ipynb) and [census-end-to-end-local-data-example.ipynb](census-end-to-end-local-data-example.ipynb):

- dataset versioning
- model registry
- endpoint deployment

# Basic Verta Setup

In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

This example features:
- **scikit-learn**'s `LinearRegression` model
- **verta** model staging via registered models
- **verta** deployment via endpoints

In [2]:
HOST = "app.verta.ai"

PROJECT_NAME = "Census Income Classification"
EXPERIMENT_NAME = "Logistic Regression"
WORKSPACE = "Demos"

In [3]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 

In [4]:
from __future__ import print_function

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

In [5]:
try:
    import wget
except ImportError:
    !pip install wget  # you may need pip3
    import wget

---

# Phase 1: Model Development

This section demonstrates logging model metadata and training artifacts to ModelDB.

## Instantiate client

In [6]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME, workspace=WORKSPACE, public_within_org=True)
expt = client.set_experiment(EXPERIMENT_NAME)

## Prepare data

In [7]:
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(train_data_url)

test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(test_data_url)

In [8]:
df_train = pd.read_csv(train_data_filename)
X_train = df_train.iloc[:,:-1]
y_train = df_train.iloc[:, -1]

df_train.head()

We will take a snapshot of our training dataset file's metadata, and version it.

In [9]:
from verta.dataset import Path

dataset = client.set_dataset(name="Census Income", workspace=WORKSPACE, public_within_org=True)
dataset_version = dataset.create_version(Path(train_data_filename))

## Prepare hyperparameters

In [10]:
hyperparam_candidates = {
    'C': [1e-6, 1e-4],
    'solver': ['lbfgs'],
    'max_iter': [15, 28],
}
hyperparam_sets = [dict(zip(hyperparam_candidates.keys(), values))
                   for values
                   in itertools.product(*hyperparam_candidates.values())]

## Train models

In [11]:
def run_experiment(hyperparams):
    # create object to track experiment run
    run = client.set_experiment_run()
    
    # create validation split
    (X_val_train, X_val_test,
     y_val_train, y_val_test) = model_selection.train_test_split(X_train, y_train,
                                                                 test_size=0.2,
                                                                 shuffle=True)

    # log hyperparameters
    run.log_hyperparameters(hyperparams)
    print(hyperparams, end=' ')
    
    # create and train model
    model = linear_model.LogisticRegression(**hyperparams)
    model.fit(X_train, y_train)
    
    # calculate and log validation accuracy
    val_acc = model.score(X_val_test, y_val_test)
    run.log_metric("val_acc", val_acc)
    print("Validation accuracy: {:.4f}".format(val_acc))
    
    # create deployment artifacts
    model_api = ModelAPI(X_train, model.predict(X_train))
    requirements = ["scikit-learn"]
    
    # save and log model
    run.log_model(model, model_api=model_api, custom_modules=[])
    run.log_requirements(requirements)
    
    # log dataset snapshot as version
    run.log_dataset_version("train", dataset_version)
    
# NOTE: run_experiment() could also be defined in a module, and executed in parallel
for hyperparams in hyperparam_sets:
    run_experiment(hyperparams)

---

# Revisit Workflow

This section demonstrates querying and retrieving runs via the Client.

## Retrieve best run

In [12]:
best_run = expt.expt_runs.sort("metrics.val_acc", descending=True)[0]
print("Validation Accuracy: {:.4f}".format(best_run.get_metric("val_acc")))

best_hyperparams = best_run.get_hyperparameters()
print("Hyperparameters: {}".format(best_hyperparams))

## Train on full dataset

In [13]:
model = linear_model.LogisticRegression(multi_class='auto', **best_hyperparams)
model.fit(X_train, y_train)

## Calculate accuracy on full training set

In [14]:
train_acc = model.score(X_train, y_train)
print("Training accuracy: {:.4f}".format(train_acc))

---

# Phase 2: Staging

## Register the best model

The best-performing model can be staged as a *registered model*, for use downstream.

In [15]:
registered_model = client.get_or_create_registered_model(name="census", workspace=WORKSPACE, public_within_org=True)

In [16]:
registered_model.create_version_from_run(best_run.id, name="v0")

---

# Phase 3: Deployment

This registered model version can be deployed to an endpoint, whereupon predictions can be made via a REST endpoint or through the client.

In [17]:
registered_model = client.get_registered_model(name="census", workspace=WORKSPACE)
model_version = registered_model.get_version(name="v0")

## Create and update an endpoint

In [18]:
endpoint = client.get_or_create_endpoint(path="/census", workspace=WORKSPACE, public_within_org=True)

In [19]:
endpoint.update(model_version, wait=True)

## Prepare "live" data

In [20]:
df_test = pd.read_csv(test_data_filename)
X_test = df_test.iloc[:,:-1]

## Query deployed model

In [21]:
model_version

In [22]:
deployed_model = endpoint.get_deployed_model()

for x in itertools.cycle(X_test.values.tolist()):
    print(deployed_model.predict([x]))
    time.sleep(.5)

---